# ROS Action

Tu zaczyna się akcja ;-). Przeznaczone do skomplikowanych działań zachowują się jak service (zadajemy cel i wreszcie dostajemy wynik) jak i topic (dostajemy sprzężenie zwrotne).

Większość akcji robota związanych jest z ruchem -- przemieszczaniem się robota w pomieszczeniu (Navigate) albo ruchem ramienia (Moveit). Robot otrzymuje zadanie -- np. dojedź do punktu i informuje o przebiegu wykonywania zadania.


Nav ma akcje zgrabnie opakowane za pomocą simple nav ale mamy wtedy komunikację synchroniczną


Akcja różni się od serwisu przez długość swojego trwania. Przez to, że nie wykonuje się od razu, z natury lepiej jest obsługiwać ją asynchronicznie plus chcemy mieć feedback z tego jak idzie zadanie

<img src="./images/action.jpg" width="30%">
Schemat wygląda tak:

- dajemy robocikowi zadanie i callbacki. Dostajemy informację o samym przyjęciu zadania jako callback
- w czasie wykonywania zadania korzysta z callbackow
- dostajemy informację o wykonaniu lub porażce zadania

In [1]:



import rclpy
from rclpy.node import Node

from std_msgs.msg import String


class MinimalNode(Node):

    def __init__(self):
        super().__init__('service_node')
    

try:
    rclpy.init()

    minimal_node = MinimalNode()
except RuntimeError:
    pass

In [14]:
#from geometry_msgs.action import 
from nav2_msgs.action import NavigateToPose
from rclpy.action import ActionClient
from geometry_msgs.msg import PoseStamped

navigate_action = ActionClient(
    minimal_node, 
    NavigateToPose,
    '/navigate_to_pose')


In [13]:
!ros2 action info navigate_to_pose

Action: navigate_to_pose
Action clients: 4
    /bt_navigator
    /waypoint_follower
    /rviz
    /_
Action servers: 1
    /bt_navigator


In [66]:


def print_feedback_goal(future):
    try:
        feedback = future.feedback
        #print("distance {}".format(feedback.distance_remaining))
        print("feedback {}".format(future.feedback))
    except:
        pass
    
def history_success(future):
    print("done")
    print("we have achieved the goal {}, {}".format(
        future.result(),
        future.result().result))
    
def received_task(future):
    goal_handle =  future.result()
    
    if not goal_handle.accepted:
        print("not accepted")
        return
    else:
        print("accepted goal")
    
    get_result_future = goal_handle.get_result_async()
    
    get_result_future.add_done_callback(history_success)


navigate_pose_goal = NavigateToPose.Goal()    
goal_pose = PoseStamped()
goal_pose.header.stamp = minimal_node.get_clock().now().to_msg()
goal_pose.pose.position.x = 4.0
goal_pose.pose.position.y = 0.0
goal_pose.pose.orientation.z = 0.7
goal_pose.pose.orientation.w = 0.71
navigate_pose_goal.pose = goal_pose
goal_future_pose = navigate_action.send_goal_async(navigate_pose_goal,feedback_callback=print_feedback_goal)
goal_future_pose.add_done_callback(received_task) 
    

accepted goal
feedback nav2_msgs.action.NavigateToPose_Feedback(current_pose=geometry_msgs.msg.PoseStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=1373, nanosec=710000000), frame_id='map'), pose=geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.09116133069743637, y=-0.04150570960887845, z=0.009303369238567689), orientation=geometry_msgs.msg.Quaternion(x=1.4214268403497265e-05, y=-9.201713734345087e-05, z=0.9777154223730775, w=-0.20993461883318748))), navigation_time=builtin_interfaces.msg.Duration(sec=0, nanosec=0), estimated_time_remaining=builtin_interfaces.msg.Duration(sec=0, nanosec=0), number_of_recoveries=0, distance_remaining=0.0)
feedback nav2_msgs.action.NavigateToPose_Feedback(current_pose=geometry_msgs.msg.PoseStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=1373, nanosec=710000000), frame_id='map'), pose=geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.09116133069743637, y=-0.04150570960887845, z=0.009303

feedback nav2_msgs.action.NavigateToPose_Feedback(current_pose=geometry_msgs.msg.PoseStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=1373, nanosec=812000000), frame_id='map'), pose=geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.09116122801032955, y=-0.041505500477942825, z=0.009283176438892957), orientation=geometry_msgs.msg.Quaternion(x=1.5830986465492956e-05, y=-8.583711478301951e-05, z=0.9777154029892152, w=-0.20993471161055313))), navigation_time=builtin_interfaces.msg.Duration(sec=0, nanosec=101000000), estimated_time_remaining=builtin_interfaces.msg.Duration(sec=0, nanosec=0), number_of_recoveries=0, distance_remaining=0.0)
feedback nav2_msgs.action.NavigateToPose_Feedback(current_pose=geometry_msgs.msg.PoseStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=1373, nanosec=812000000), frame_id='map'), pose=geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.09116122801032955, y=-0.041505500477942825, z=0.0092831764

feedback nav2_msgs.action.NavigateToPose_Feedback(current_pose=geometry_msgs.msg.PoseStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=1373, nanosec=948000000), frame_id='map'), pose=geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.09116102665311332, y=-0.0415059240555567, z=0.009282048897411594), orientation=geometry_msgs.msg.Quaternion(x=8.606524852881642e-06, y=-9.504134322387478e-05, z=0.9777153951516211, w=-0.20993474456740202))), navigation_time=builtin_interfaces.msg.Duration(sec=0, nanosec=224000000), estimated_time_remaining=builtin_interfaces.msg.Duration(sec=0, nanosec=0), number_of_recoveries=0, distance_remaining=0.0)
feedback nav2_msgs.action.NavigateToPose_Feedback(current_pose=geometry_msgs.msg.PoseStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=1373, nanosec=948000000), frame_id='map'), pose=geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.09116102665311332, y=-0.0415059240555567, z=0.009282048897411

feedback nav2_msgs.action.NavigateToPose_Feedback(current_pose=geometry_msgs.msg.PoseStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=1374, nanosec=50000000), frame_id='map'), pose=geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.09116122030109586, y=-0.041505419450923786, z=0.009283660861020147), orientation=geometry_msgs.msg.Quaternion(x=1.5727066193952633e-05, y=-8.385605364759309e-05, z=0.9777153710349407, w=-0.20993486123755148))), navigation_time=builtin_interfaces.msg.Duration(sec=0, nanosec=339000000), estimated_time_remaining=builtin_interfaces.msg.Duration(sec=0, nanosec=0), number_of_recoveries=1, distance_remaining=0.0)
feedback nav2_msgs.action.NavigateToPose_Feedback(current_pose=geometry_msgs.msg.PoseStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=1374, nanosec=84000000), frame_id='map'), pose=geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.09116142801943483, y=-0.041505427771339753, z=0.009282772353

feedback nav2_msgs.action.NavigateToPose_Feedback(current_pose=geometry_msgs.msg.PoseStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=1374, nanosec=220000000), frame_id='map'), pose=geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.09116123140962107, y=-0.04150566368054415, z=0.0093072821228047), orientation=geometry_msgs.msg.Quaternion(x=1.0924420433217596e-05, y=-9.060217351384083e-05, z=0.9777153569266243, w=-0.20993492444504544))), navigation_time=builtin_interfaces.msg.Duration(sec=0, nanosec=472000000), estimated_time_remaining=builtin_interfaces.msg.Duration(sec=0, nanosec=0), number_of_recoveries=1, distance_remaining=0.0)
feedback nav2_msgs.action.NavigateToPose_Feedback(current_pose=geometry_msgs.msg.PoseStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=1374, nanosec=220000000), frame_id='map'), pose=geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.09116123140962107, y=-0.04150566368054415, z=0.00930728212280

...

feedback nav2_msgs.action.NavigateToPose_Feedback(current_pose=geometry_msgs.msg.PoseStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=1375, nanosec=206000000), frame_id='map'), pose=geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.09126665890231278, y=-0.041447431633392484, z=0.00928034723231903), orientation=geometry_msgs.msg.Quaternion(x=4.547041051026371e-05, y=-0.0002635038197714694, z=0.9705126769609859, w=-0.24104993747395953))), navigation_time=builtin_interfaces.msg.Duration(sec=1, nanosec=500000000), estimated_time_remaining=builtin_interfaces.msg.Duration(sec=0, nanosec=0), number_of_recoveries=2, distance_remaining=0.0)
feedback nav2_msgs.action.NavigateToPose_Feedback(current_pose=geometry_msgs.msg.PoseStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=1375, nanosec=206000000), frame_id='map'), pose=geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.09126665890231278, y=-0.041447431633392484, z=0.009280347232

feedback nav2_msgs.action.NavigateToPose_Feedback(current_pose=geometry_msgs.msg.PoseStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=1375, nanosec=308000000), frame_id='map'), pose=geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.0913458071907991, y=-0.04140088399557473, z=0.009297631779386452), orientation=geometry_msgs.msg.Quaternion(x=8.659424955865584e-05, y=-0.00045451341854946427, z=0.9644616391511928, w=-0.26422250571210393))), navigation_time=builtin_interfaces.msg.Duration(sec=1, nanosec=575000000), estimated_time_remaining=builtin_interfaces.msg.Duration(sec=0, nanosec=0), number_of_recoveries=2, distance_remaining=0.0)
feedback nav2_msgs.action.NavigateToPose_Feedback(current_pose=geometry_msgs.msg.PoseStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=1375, nanosec=308000000), frame_id='map'), pose=geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.0913458071907991, y=-0.04140088399557473, z=0.00929763177938

feedback nav2_msgs.action.NavigateToPose_Feedback(current_pose=geometry_msgs.msg.PoseStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=1375, nanosec=444000000), frame_id='map'), pose=geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.09146818876869833, y=-0.04131241038668787, z=0.009302985813726638), orientation=geometry_msgs.msg.Quaternion(x=0.00012570640964894333, y=-0.0006471009023177827, z=0.9532110721798249, w=-0.30230484172786487))), navigation_time=builtin_interfaces.msg.Duration(sec=1, nanosec=718000000), estimated_time_remaining=builtin_interfaces.msg.Duration(sec=0, nanosec=0), number_of_recoveries=2, distance_remaining=0.0)
feedback nav2_msgs.action.NavigateToPose_Feedback(current_pose=geometry_msgs.msg.PoseStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=1375, nanosec=444000000), frame_id='map'), pose=geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.09146818876869833, y=-0.04131241038668787, z=0.009302985813

feedback nav2_msgs.action.NavigateToPose_Feedback(current_pose=geometry_msgs.msg.PoseStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=1375, nanosec=512000000), frame_id='map'), pose=geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.0915275622467604, y=-0.04126538527371393, z=0.009301671452287285), orientation=geometry_msgs.msg.Quaternion(x=0.00015619483332145987, y=-0.0007846305407937798, z=0.946964625354758, w=-0.32133682995388396))), navigation_time=builtin_interfaces.msg.Duration(sec=1, nanosec=826000000), estimated_time_remaining=builtin_interfaces.msg.Duration(sec=0, nanosec=0), number_of_recoveries=2, distance_remaining=0.0)
feedback nav2_msgs.action.NavigateToPose_Feedback(current_pose=geometry_msgs.msg.PoseStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=1375, nanosec=512000000), frame_id='map'), pose=geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.0915275622467604, y=-0.04126538527371393, z=0.009301671452287

feedback nav2_msgs.action.NavigateToPose_Feedback(current_pose=geometry_msgs.msg.PoseStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=1375, nanosec=614000000), frame_id='map'), pose=geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.09158477024066758, y=-0.041214631653281375, z=0.009301622737238092), orientation=geometry_msgs.msg.Quaternion(x=0.00017680327475730383, y=-0.0009117221140269012, z=0.9401465313412148, w=-0.34076918450832794))), navigation_time=builtin_interfaces.msg.Duration(sec=1, nanosec=894000000), estimated_time_remaining=builtin_interfaces.msg.Duration(sec=0, nanosec=0), number_of_recoveries=2, distance_remaining=0.0)
feedback nav2_msgs.action.NavigateToPose_Feedback(current_pose=geometry_msgs.msg.PoseStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=1375, nanosec=614000000), frame_id='map'), pose=geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.09158477024066758, y=-0.041214631653281375, z=0.0093016227

In [24]:
import subscription_threaded
threaded_spinner = subscription_threaded.ThreadedSpinner(minimal_node)

In [71]:
# możemy albo w osobnym wątku albo spin_until_future_complete(minimal_node, send_goal_future)


threaded_spinner.spin_in_thread()

feedback nav2_msgs.action.NavigateToPose_Feedback(current_pose=geometry_msgs.msg.PoseStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=1408, nanosec=118000000), frame_id='map'), pose=geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=-0.0795978485511517, y=0.054837724023454826, z=0.010016145546093112), orientation=geometry_msgs.msg.Quaternion(x=-0.0009983647298232605, y=-0.0008581720648857926, z=0.7990599291633425, w=0.6012499450428642))), navigation_time=builtin_interfaces.msg.Duration(sec=34, nanosec=430000000), estimated_time_remaining=builtin_interfaces.msg.Duration(sec=0, nanosec=0), number_of_recoveries=18, distance_remaining=0.0)
done
we have achieved the goal nav2_msgs.action.NavigateToPose_GetResult_Response(status=6, result=nav2_msgs.action.NavigateToPose_Result(result=std_msgs.msg.Empty())), nav2_msgs.action.NavigateToPose_Result(result=std_msgs.msg.Empty())


In [35]:
goal_handle = goal_future_pose.result()

feedback nav2_msgs.action.NavigateToPose_Feedback(current_pose=geometry_msgs.msg.PoseStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=1007, nanosec=802000000), frame_id='map'), pose=geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.15472042182394175, y=-0.11323087557267564, z=0.00956680696841234), orientation=geometry_msgs.msg.Quaternion(x=1.1518271237683544e-05, y=-5.455888634071492e-05, z=0.9014789172273856, w=-0.4328230108083416))), navigation_time=builtin_interfaces.msg.Duration(sec=3, nanosec=711000000), estimated_time_remaining=builtin_interfaces.msg.Duration(sec=0, nanosec=0), number_of_recoveries=2, distance_remaining=0.0)
feedback nav2_msgs.action.NavigateToPose_Feedback(current_pose=geometry_msgs.msg.PoseStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=1007, nanosec=802000000), frame_id='map'), pose=geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.15472042182394175, y=-0.11323087557267564, z=0.00956680696841

feedback nav2_msgs.action.NavigateToPose_Feedback(current_pose=geometry_msgs.msg.PoseStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=1007, nanosec=904000000), frame_id='map'), pose=geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.15472113457029169, y=-0.11323017854175055, z=0.009594790554770718), orientation=geometry_msgs.msg.Quaternion(x=3.393353044807652e-05, y=-4.708637490658799e-05, z=0.901478940089362, w=-0.4328229628921614))), navigation_time=builtin_interfaces.msg.Duration(sec=3, nanosec=808000000), estimated_time_remaining=builtin_interfaces.msg.Duration(sec=0, nanosec=0), number_of_recoveries=2, distance_remaining=0.0)
feedback nav2_msgs.action.NavigateToPose_Feedback(current_pose=geometry_msgs.msg.PoseStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=1007, nanosec=904000000), frame_id='map'), pose=geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.15472113457029169, y=-0.11323017854175055, z=0.009594790554770

feedback nav2_msgs.action.NavigateToPose_Feedback(current_pose=geometry_msgs.msg.PoseStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=1007, nanosec=972000000), frame_id='map'), pose=geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.15472101030050833, y=-0.11323023466379567, z=0.009596869785829232), orientation=geometry_msgs.msg.Quaternion(x=3.0255823979780773e-05, y=-4.707733393280281e-05, z=0.9014788931754376, w=-0.4328230608776499))), navigation_time=builtin_interfaces.msg.Duration(sec=3, nanosec=893000000), estimated_time_remaining=builtin_interfaces.msg.Duration(sec=0, nanosec=0), number_of_recoveries=2, distance_remaining=0.0)
feedback nav2_msgs.action.NavigateToPose_Feedback(current_pose=geometry_msgs.msg.PoseStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=1008, nanosec=6000000), frame_id='map'), pose=geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.15472086467383317, y=-0.11323043806691924, z=0.009591667647191

feedback nav2_msgs.action.NavigateToPose_Feedback(current_pose=geometry_msgs.msg.PoseStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=1008, nanosec=74000000), frame_id='map'), pose=geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.1547208283389261, y=-0.11323056289565259, z=0.009591506788738314), orientation=geometry_msgs.msg.Quaternion(x=2.3455198760225904e-05, y=-5.2034435643255706e-05, z=0.9014788441620678, w=-0.43282316281651734))), navigation_time=builtin_interfaces.msg.Duration(sec=3, nanosec=985000000), estimated_time_remaining=builtin_interfaces.msg.Duration(sec=0, nanosec=0), number_of_recoveries=2, distance_remaining=0.0)
feedback nav2_msgs.action.NavigateToPose_Feedback(current_pose=geometry_msgs.msg.PoseStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=1008, nanosec=74000000), frame_id='map'), pose=geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.1547208283389261, y=-0.11323056289565259, z=0.009591506788738

feedback nav2_msgs.action.NavigateToPose_Feedback(current_pose=geometry_msgs.msg.PoseStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=1008, nanosec=142000000), frame_id='map'), pose=geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.15544260195761028, y=-0.11232984734104745, z=0.009557753732370767), orientation=geometry_msgs.msg.Quaternion(x=-4.582549663645237e-06, y=-6.558156472544427e-05, z=0.9014788087267053, w=-0.43282323539146894))), navigation_time=builtin_interfaces.msg.Duration(sec=4, nanosec=71000000), estimated_time_remaining=builtin_interfaces.msg.Duration(sec=0, nanosec=0), number_of_recoveries=2, distance_remaining=0.0)
feedback nav2_msgs.action.NavigateToPose_Feedback(current_pose=geometry_msgs.msg.PoseStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=1008, nanosec=142000000), frame_id='map'), pose=geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.15544260195761028, y=-0.11232984734104745, z=0.0095577537323

...

feedback nav2_msgs.action.NavigateToPose_Feedback(current_pose=geometry_msgs.msg.PoseStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=1008, nanosec=312000000), frame_id='map'), pose=geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.1586327508144818, y=-0.10834904833496772, z=0.009556993939278773), orientation=geometry_msgs.msg.Quaternion(x=1.9773293922321508e-05, y=-5.7589842647786854e-05, z=0.9014788135060854, w=-0.43282322614675767))), navigation_time=builtin_interfaces.msg.Duration(sec=4, nanosec=208000000), estimated_time_remaining=builtin_interfaces.msg.Duration(sec=0, nanosec=0), number_of_recoveries=2, distance_remaining=0.0)
feedback nav2_msgs.action.NavigateToPose_Feedback(current_pose=geometry_msgs.msg.PoseStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=1008, nanosec=312000000), frame_id='map'), pose=geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.1586327508144818, y=-0.10834904833496772, z=0.0095569939392

feedback nav2_msgs.action.NavigateToPose_Feedback(current_pose=geometry_msgs.msg.PoseStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=1008, nanosec=414000000), frame_id='map'), pose=geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.1596962114828706, y=-0.1070216760046061, z=0.009545940115201549), orientation=geometry_msgs.msg.Quaternion(x=3.8240488802872113e-05, y=-4.424774541179163e-05, z=0.9014787599086818, w=-0.4328233381107209))), navigation_time=builtin_interfaces.msg.Duration(sec=4, nanosec=310000000), estimated_time_remaining=builtin_interfaces.msg.Duration(sec=0, nanosec=0), number_of_recoveries=2, distance_remaining=0.0)
feedback nav2_msgs.action.NavigateToPose_Feedback(current_pose=geometry_msgs.msg.PoseStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=1008, nanosec=414000000), frame_id='map'), pose=geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.1596962114828706, y=-0.1070216760046061, z=0.00954594011520154

feedback nav2_msgs.action.NavigateToPose_Feedback(current_pose=geometry_msgs.msg.PoseStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=1008, nanosec=516000000), frame_id='map'), pose=geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.16041927210212248, y=-0.1061194105918063, z=0.00953034836135647), orientation=geometry_msgs.msg.Quaternion(x=2.7880250838021043e-05, y=-4.948568027438558e-05, z=0.9014788065496833, w=-0.4328232411916182))), navigation_time=builtin_interfaces.msg.Duration(sec=4, nanosec=392000000), estimated_time_remaining=builtin_interfaces.msg.Duration(sec=0, nanosec=0), number_of_recoveries=2, distance_remaining=0.0)
feedback nav2_msgs.action.NavigateToPose_Feedback(current_pose=geometry_msgs.msg.PoseStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=1008, nanosec=516000000), frame_id='map'), pose=geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.16041927210212248, y=-0.1061194105918063, z=0.0095303483613564

feedback nav2_msgs.action.NavigateToPose_Feedback(current_pose=geometry_msgs.msg.PoseStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=1008, nanosec=618000000), frame_id='map'), pose=geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.16254432444455724, y=-0.1034675372065651, z=0.00950995053420092), orientation=geometry_msgs.msg.Quaternion(x=-5.2444102815473625e-06, y=-6.568679650861309e-05, z=0.9014787728301253, w=-0.4328233101329368))), navigation_time=builtin_interfaces.msg.Duration(sec=4, nanosec=488000000), estimated_time_remaining=builtin_interfaces.msg.Duration(sec=0, nanosec=0), number_of_recoveries=2, distance_remaining=0.0)
feedback nav2_msgs.action.NavigateToPose_Feedback(current_pose=geometry_msgs.msg.PoseStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=1008, nanosec=618000000), frame_id='map'), pose=geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.16254432444455724, y=-0.1034675372065651, z=0.009509950534200

feedback nav2_msgs.action.NavigateToPose_Feedback(current_pose=geometry_msgs.msg.PoseStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=1008, nanosec=720000000), frame_id='map'), pose=geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.16433046863061684, y=-0.10123847904121341, z=0.009479858113216652), orientation=geometry_msgs.msg.Quaternion(x=1.293230294829944e-05, y=-5.545095983510187e-05, z=0.901478753308178, w=-0.4328233520639567))), navigation_time=builtin_interfaces.msg.Duration(sec=4, nanosec=574000000), estimated_time_remaining=builtin_interfaces.msg.Duration(sec=0, nanosec=0), number_of_recoveries=2, distance_remaining=0.0)
feedback nav2_msgs.action.NavigateToPose_Feedback(current_pose=geometry_msgs.msg.PoseStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=1008, nanosec=720000000), frame_id='map'), pose=geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.16433046863061684, y=-0.10123847904121341, z=0.009479858113216

In [ ]:
send_goal_future = self.nav_to_pose_client.send_goal_async(goal_msg,
                                                                   self._feedbackCallback)
rclpy.spin_until_future_complete(self, send_goal_future)
self.goal_handle = send_goal_future.result()

In [58]:
goal_handle.cancel_goal_async()

In [68]:
threaded_spinner.stop()

feedback nav2_msgs.action.NavigateToPose_Feedback(current_pose=geometry_msgs.msg.PoseStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=1388, nanosec=160000000), frame_id='map'), pose=geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.0173561971247842, y=0.09409956551050336, z=0.00922212717337724), orientation=geometry_msgs.msg.Quaternion(x=-0.00010021150898785857, y=5.7776588526754515e-05, z=-0.5273066174665932, w=0.8496750660078588))), navigation_time=builtin_interfaces.msg.Duration(sec=14, nanosec=466000000), estimated_time_remaining=builtin_interfaces.msg.Duration(sec=0, nanosec=0), number_of_recoveries=10, distance_remaining=0.0)
feedback nav2_msgs.action.NavigateToPose_Feedback(current_pose=geometry_msgs.msg.PoseStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=1388, nanosec=160000000), frame_id='map'), pose=geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.0173561971247842, y=0.09409956551050336, z=0.009222127173377

In [70]:
rclpy.spin_until_future_complete(minimal_node, goal_future_pose)

print(goal_future_pose.result())

ClientGoalHandle <id=[ 44  27 101 209  26  83  69 215 185  25 244 153 101 167 169 105], accepted=True, status=2>


In [72]:
# rzeczy z feedbacku możemy odczytywać także jako topic
minimal_node.get_topic_names_and_types()



[('/NavigateToPose/_action/feedback',
  ['nav2_msgs/action/NavigateToPose_FeedbackMessage']),
 ('/NavigateToPose/_action/status', ['action_msgs/msg/GoalStatusArray']),
 ('/amcl/transition_event', ['lifecycle_msgs/msg/TransitionEvent']),
 ('/amcl_pose', ['geometry_msgs/msg/PoseWithCovarianceStamped']),
 ('/assisted_teleop/_action/feedback',
  ['nav2_msgs/action/AssistedTeleop_FeedbackMessage']),
 ('/assisted_teleop/_action/status', ['action_msgs/msg/GoalStatusArray']),
 ('/backup/_action/feedback', ['nav2_msgs/action/BackUp_FeedbackMessage']),
 ('/backup/_action/status', ['action_msgs/msg/GoalStatusArray']),
 ('/behavior_server/transition_event', ['lifecycle_msgs/msg/TransitionEvent']),
 ('/behavior_tree_log', ['nav2_msgs/msg/BehaviorTreeLog']),
 ('/bond', ['bond/msg/Status']),
 ('/bt_navigator/transition_event', ['lifecycle_msgs/msg/TransitionEvent']),
 ('/camera/camera_info', ['sensor_msgs/msg/CameraInfo']),
 ('/camera/image_raw', ['sensor_msgs/msg/Image']),
 ('/camera/image_raw/compr

[Next exercise: 8. Parameters](8.%20Parameters.ipynb)